<font color="magenta" size=7>Source Code

This is the source code that runs the `spikesort_easy` application.

<font color="red">This code is based on [this repository](https://github.com/flatironinstitute/spikeforest2)

# Notebook Setup

In [ ]:
#!git login

In [ ]:
#!git clone https://github.com/braingeneers/dashboard ./dashboard

In [ ]:
from dashboard.apps.utils import NumpyS3Memmap

In [ ]:
#!pip install tenacity
#!pip install smart_open

## Import Stuff

In [ ]:
import kachery as ka
import os
import subprocess
import sys
import numpy as np
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
#do things to get braingeneers data
#install("git+https://github.com/braingeneers/braingeneerspy.git")
#install("matplotlib")
import hither_sf as hither
from spikeforest2 import sorters
import matplotlib
import braingeneers.datasets_electrophysiology
import spikeinterface.extractors as se #for mda conversion only
import spiketoolkit as st
import braingeneers
from braingeneers import datasets_electrophysiology as ephys
import numpy as np
import json
import spikeforest2_utils
from spikeforest2_utils import AutoRecordingExtractor, MdaRecordingExtractor
import hither_sf as hither
import kachery as ka
import io
import sys
spikeforest2_utils.__version__=1.0

## Create Directories

In [ ]:
folders = !ls
if "kach_dir" not in folders:
    !mkdir kach_dir
    !touch kach_dir/raw.mda

In [ ]:
current_dir = ! pwd
kach_dir = current_dir[0] + "/kach_dir/"

# <font color="blue">Choose Experiment, time range, channels 

## <font color="blue">choose time range, experiment name, experiment num

In [ ]:
fname = os.environ['KACHERY_STORAGE_DIR']=kach_dir
#batch_uuid = '2020-11-27-e-UCSF-axionplate' 
#experiment_num= 9 #arpitha make this a button to choose exp num
#start = 1 #arpitha, sliders here for start minutes
#stop = 5 #and end minutes
#recording_path = 'sha1://961f4a641af64dded4821610189f808f0192de4d/SYNTH_MEAREC_TETRODE/synth_mearec_tetrode_noise10_K10_C4/002_synth.json'

In [ ]:
!aws --endpoint https://s3.nautilus.optiputer.net s3 ls s3://braingeneersdev/ephys/2020-11-27-e-primary-axion-Afternoon1/numpy/

In [ ]:
!aws --endpoint https://s3.nautilus.optiputer.net s3 ls s3://braingeneersdev/ephys/2020-11-27-e-primary-axion-Afternoon1/numpy/well_A1_chan_group_idx_3_time_000.npy

In [ ]:
this_numpy = "s3://braingeneersdev/ephys/2020-11-27-e-primary-axion-Afternoon1/numpy/well_A2_chan_group_idx_10_time_001.npy"

In [ ]:
#!export ENDPOINT_URL=https://s3.nautilus.optiputer.net

In [ ]:
os.environ['ENDPOINT_URL'] = "https://s3.nautilus.optiputer.net"

In [ ]:
import numpy as np
import os
import functools
from smart_open import open
open = functools.partial(open, transport_params={'resource_kwargs':{'endpoint_url': os.environ['ENDPOINT_URL']}})  # wraps open setting a default endpoint_url
#data = NumpyS3Memmap('s3://braingeneersdev/dfparks/test/test.npy')
data = NumpyS3Memmap(this_numpy)


In [ ]:
data.shape

In [ ]:
data[0, :]

In [ ]:
#data = data.T

In [ ]:
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

tracemalloc.start()

In [ ]:
# Example using open now works with the PRP exactly the same way it does if the file is local
#with open('s3://braingeneersdev/dfparks/test/hello.txt', 'r') as f:
#with open(this_numpy, 'r') as f:
#    print("sh")
#Hello World!
# Example using open with a local file, identical to the S3 case
#with open('/tmp/hello.txt', 'r') as f:
#    print(f.read())

In [ ]:
#from apps.utils import NumpyS3Memmap

In [ ]:
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

## view other experiment numbers in the same batch

## load data, time array (seconds), and sampling freq

In [ ]:
#X, t, fs = ephys.load_blocks(batch_uuid, experiment_num, start=start, stop=stop)
X = data
X.shape


In [ ]:
fs = 22000 #????
#X = X.T #spikeinterface wants it MxN (channel number by time)
X.shape

Y = np.append(X[:,0:1], X[:,1:2], axis=1)
#Z = np.append(X[:,2:3], X[:,3:4], axis=1)
Y = Y.T

#M = np.append(Y,Z,axis=1)

In [ ]:
#M = M.T

In [ ]:
Y.shape

In [ ]:
X = Y

In [ ]:
X.shape

In [ ]:
print("this is sampling Freq: ", fs)

## <font color="blue">optionally (ideally): apply geometry for locations of electrodes

In [ ]:
# last_chan = X[selected_channels[0],:]
# print("last_chan shape ", last_chan.shape)
# print("X[0].shape ", X[0].shape)


#geom = np.zeros((X.shape[1],2))
#geom = np.zeros((1,2)) # initialize recording device geometry to zeros

# if len(selected_channels)==1:
#     this_chan = X[selected_channels[0]]
#     this_chan = np.array([this_chan])
#     X = this_chan
#     print("1 chan: ", X.shape)
    
    
# if len(selected_channels)==2:
#     X = np.append([X[selected_channels[0]]], [X[selected_channels[1]]], axis=0)
#     print("2 chan: ", X.shape)
#     geom = np.array([[0.0,0.0],[0.0,1.0]])

# if len(selected_channels)>2: #PROBLEMATIC
#     for chan in range(1,len(selected_channels)):
#         last_chan = np.append([last_chan],[X[selected_channels[chan]]],axis=0)
#     X = last_chan


In [ ]:
X.shape

In [ ]:
#print(geom)

In [ ]:
#X.shape
#X[1:10,1:10]
print("minutes = ", X.shape[0]/fs/60)
#print("expected minutes =", stop-start)

In [ ]:
if X.shape[0] == 2 :
    geom = np.array([[0.0,0.0],[0.0,1.0]])
    print("only two channels, so Geom always two points 1 unit apart.")

In [ ]:
#geom.shape #the 2d locations in space of each channel on the recording device

# Create "Recording" for spikeForest and Filter recording

In [ ]:
recording=se.NumpyRecordingExtractor(
    timeseries=X,
    geom=geom,
    sampling_frequency=fs
)

In [ ]:
X

In [ ]:
#recording = st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000)

In [ ]:
fname

raw_path = ka.store_file(fname  + 'raw.mda')

kach_dir_files = ! ls kach_dir
if "geom.csv" not in kach_dir_files :
    ! touch kach_dir/geom.csv

obj = dict(
    raw=raw_path,
    params=ka.load_object(fname + 'params.json'),
    geom=np.genfromtxt(ka.load_file(fname + 'geom.csv'), delimiter=',').tolist()
)

obj['self_reference'] = ka.store_object(obj, basename='{}.json'.format(label))
print(obj['self_reference'])
with open(output_fname, 'w') as f:
    json.dump(obj, f, indent=4)

In [ ]:
def register_recording(*, fname, recording, output_fname, label):
            raw_path = ka.store_file(fname  + 'raw.mda')
            obj = dict(
                raw=raw_path,
                params=ka.load_object(fname + 'params.json'),
                #geom=np.genfromtxt(ka.load_file(fname + 'geom.csv'), delimiter=',').tolist()
            )
            obj['self_reference'] = ka.store_object(obj, basename='{}.json'.format(label))

            with open(output_fname, 'w') as f: json.dump(obj, f, indent=4)
            return obj['self_reference']
            
            

In [ ]:
recording_path =  register_recording( recording=recording, fname=fname, output_fname=fname+'new_recording.json', label='new_recording')

# <font color="black"> Sort

In [ ]:
# dummy dataset courtesy of spikeforest's example 
#recording_path = 'sha1://961f4a641af64dded4821610189f808f0192de4d/SYNTH_MEAREC_TETRODE/synth_mearec_tetrode_noise10_K10_C4/002_synth.json' 


In [ ]:
with ka.config(fr='default_readonly'):
    #with hither.config(cache='default_readwrite'):
        with hither.config(container='default'):
            result = sorters.spykingcircus.run(
                    recording_path=recording_path,
                    sorting_out=hither.File()
                    )
            print("recording out path: ", recording_path)
            print("sorting out path: ",result.outputs.sorting_out)
            sorting_path = str(result.outputs.sorting_out)

print(result.outputs.sorting_out)


In [ ]:
sorting_out_short = sorting_path[12:-1]

print("This is the new short path for the sorting: ",sorting_out_short)

In [ ]:
from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka

universal_sorting_path = ka.store_file(sorting_out_short)

recording = AutoRecordingExtractor(recording_path, download=False)
sorting_true = AutoSortingExtractor(universal_sorting_path)

In [ ]:
# Now you can access the recording using the SpikeInterface API
# For example, print the electrode locations and unit IDs
print(recording.get_channel_locations())
print(sorting_true.get_unit_ids())

In [ ]:
import pickle

In [ ]:
import spikeinterface.widgets as sw

# Make Plots

In [ ]:
w_ts = sw.plot_timeseries(recording)
    
    #does not work w no geom:
    #w_el = sw.plot_electrode_geometry(recording)
    #pickle.dump(w_el, output, pickle.HIGHEST_PROTOCOL)
    
w_sp = sw.plot_spectrum(recording)
    
w_rs = sw.plot_rasters(sorting_true, sampling_frequency=fs)
    
w_isi = sw.plot_isi_distribution(sorting_true, sampling_frequency=fs, bins=10, window=1)
    
w_ach = sw.plot_autocorrelograms(sorting_true, sampling_frequency=fs, bin_size=1, window=10, unit_ids=[1, 2, 4, 5, 8, 10, 7])
    
w_cch = sw.plot_crosscorrelograms(sorting_true, sampling_frequency=fs, unit_ids=[1, 5, 8], bin_size=0.1, window=5)

w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=sorting_true.get_unit_ids(),  max_spikes_per_unit=100)
    
w_ampd = sw.plot_amplitudes_distribution(recording, sorting_true, max_spikes_per_unit=300)
    
w_ampt = sw.plot_amplitudes_timeseries(recording, sorting_true, max_spikes_per_unit=300)
    

# Curate Units

In [ ]:
curated_units = [0,1]

In [ ]:
import matplotlib.pyplot as plt

w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=curated_units,max_channels=1,color='darkgreen', max_spikes_per_unit=300)#,figure=fig)#,axis=)

plt.yticks(np.arange(-1.5, 2.0, 1))
print("Each graph spans 3ms x axis. Y axis is microvolts -- the same as the amplitude distributions in the other graphs")
w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=[curated_units[1],curated_units[0]],max_channels=1,color='darkgreen', max_spikes_per_unit=300)#,figure=fig)#,axis=)

plt.yticks(np.arange(-1.5, 2.0, 1))

# Remove Containers

In [ ]:
!sudo docker ps -a | grep Exit | cut -d ' ' -f 1 | xargs sudo docker rm

In [ ]:
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

tracemalloc.start()